In [1]:
import pandas as pd

In [14]:
def is_offensive(cls):
    if cls == 1: return 1
    else: return 0

def is_hate(cls):
    if cls == 0: return 1
    else: return 0

df = pd.read_csv('../twitter/labeled_data-davidson.csv')
df.drop(['count', 'neither', 'Unnamed: 0'], axis=1, inplace=True)
# df.head()
df['is_offensive'] = df['class'].apply(is_offensive)
df['is_hate'] = df['class'].apply(is_hate)


In [15]:
import re
import unicodedata
import contractions
import string
import nltk

def remove_accented_chars(text):
    new_text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    return new_text

def remove_special_characters(text):
    # define the pattern to keep
    pat = r'[^a-zA-z0-9.,!?/:;\"\'\s]'
    return re.sub(pat, '', text)

def remove_punctuation(text):
    text = ''.join([c for c in text if c not in string.punctuation])
    return text

def stemming(text):
    stemmer = nltk.porter.PorterStemmer()
    text = ' '.join([stemmer.stem(word) for word in text.split()])
    return text

def remove_extra_whitespace_tabs(text):
    #pattern = r'^\s+$|\s+$'
    pattern = r'^\s*|\s\s*'
    return re.sub(pattern, ' ', text).strip()

def clean_text(text):
    cleaned_text = re.sub(
        "(^!+)|([@][^\s]*[:]?)|(&#\d+;)|(https?[^\s]*)|(&amp;)|(RT)|(&lt;)|(\n)",
        "",
        text
    )

    # removing accented characters
    cleaned_text = remove_accented_chars(cleaned_text)

    # removing special characters
    cleaned_text = remove_special_characters(cleaned_text)

    # removing contractions
    cleaned_text = contractions.fix(cleaned_text)

    # removing punctuation
    cleaned_text = remove_punctuation(cleaned_text)

    # text stemming
    cleaned_text = stemming(cleaned_text)

    # removing extra whitespaces
    cleaned_text = cleaned_text.strip()
    cleaned_text = remove_extra_whitespace_tabs(cleaned_text)

    # print(text)
    # print(cleaned_text)
    # print()

    return cleaned_text

df['tweet'] = df['tweet'].apply(clean_text)
df.to_csv('../twitter/twitter_preprocessed.csv')